# グレーコード

グレーコードとは、隣接する値同士が1ビットだけ異なるように設計された数列であり、主に誤り訂正、エンコーディング、回路設計などで利用されます。一般的な二進数と異なり、1つの値から次の値への変化が最小限に抑えられるため、アナログ-デジタル変換などの用途でも重要な役割を果たします。

In [2]:
from z3 import *

## Int変数で解く

以下はZ3を用いてNビットのグレーコードを生成するプログラムです。連続する2つのコードの差分が2のべき乗であるという制約条件を加えることで、グレーコードの特性を満たすようにしています。

In [3]:
def gray_code(nbits):
    n = 2 ** nbits
    codes = IntVector('code', n)
    solver = Solver()
    
    for c in codes:
        solver.add(0 <= c, c < n)
        
    solver.add(Distinct(codes))
    
    for c1, c2 in zip(codes, codes[1:] + codes[:1]):
        diff = Abs(c1 - c2)
        solver.add(Or([diff == 2**bit for bit in range(nbits)]))
    
    solver.add(codes[0] == 0)
    solver.check()
    model = solver.model()
    return [model[c].as_long() for c in codes]

nbits = 5
%time codes = gray_code(nbits)
for c in codes:
    print(f'{c:0{nbits}b}')

CPU times: total: 5.17 s
Wall time: 5.47 s
00000
00010
10010
11010
11011
11111
11110
10110
10100
11100
11101
10101
10011
10111
11000
11001
10001
00001
00011
00100
00101
00110
00111
01000
01001
01010
01011
01100
01101
01110
01111
10000


## BitVec変数で加速

上記のコードは正しくグレーコードを生成できますが、整数 (`Int`) を使っているため、Z3 の制約解決が遅くなります。そこで、次に `BitVec` を用いてより効率的にグレーコードを生成する方法を示します。

`Int` の代わりに `BitVec` を使用し、前後の2つのコードの差分を XOR 演算子 (`^`) で求めます。グレーコードの隣接する値は1ビットだけ異なるため、この差分が「1が1つだけ立っている」ことを判定する必要があります。そのため、以下の条件を追加します。

```python
diff = c1 ^ c2
solver.add(diff & (diff - 1) == 0, diff != 0)
```

これは、`diff` が 2 のべき乗であることを確認するための一般的なビット演算テクニックです。

In [5]:
def gray_code_bv(nbits):
    codes = [BitVec(f'code_{i}', nbits) for i in range(2**nbits)]
    solver = Solver()
    solver.add(Distinct(codes))
    
    for c1, c2 in zip(codes, codes[1:] + codes[:1]):
        diff = c1 ^ c2
        solver.add(diff & (diff - 1) == 0, diff != 0)
    
    solver.add(codes[0] == 0)
    
    solver.check()
    model = solver.model()
    return [model[c].as_long() for c in codes]

この方法では、`BitVec` を使用することで Z3 の処理が効率化され、整数 (`Int`) を使った場合と比べて数十倍高速にグレーコードを求めることができます。

In [7]:
%time codes = gray_code_bv(nbits)
for c in codes:
    print(f'{c:0{nbits}b}')

CPU times: total: 78.1 ms
Wall time: 81.7 ms
00000
00010
10010
10011
10001
10000
11000
11001
11101
11111
10111
10101
10100
10110
00110
00100
01100
11100
11110
01110
01111
00111
00011
01011
11011
11010
01010
01000
01001
01101
00101
00001
